<a href="https://colab.research.google.com/github/chehak2004/Stock-Tracker/blob/main/Stock_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages if not already installed (uncomment if needed)
!pip install yfinance ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time
import yfinance as yf
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00


In [ ]:
# -----------------------------
# Email Configuration Settings
# -----------------------------
# Replace these with your email settings.
EMAIL_ADDRESS = "chehaktrehan21@gmail.com"      # Your sender email address
EMAIL_PASSWORD = "njwd obky invw kfjm"          # Your email password or app-specific password
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# -----------------------------
# Email Notification Function
# -----------------------------
def send_email_notification(to_email, stock_symbol, current_price, condition):
    """
    Sends an email notification.
    :param to_email: Recipient email address.
    :param stock_symbol: The stock symbol.
    :param current_price: The price at which the condition was met.
    :param condition: A string indicating whether it was a "TARGET" or "STOP LOSS" hit.
    """
    subject = f"Stock Alert: {stock_symbol} has hit {condition}!"
    body = (f"Hello,\n\n"
            f"This is an automated alert that {stock_symbol} has reached a price of {current_price:.2f},\n"
            f"which meets the {condition} condition that you set.\n\n"
            f"Regards,\nStock Alert System")

    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", str(e))


# -----------------------------
# Stock Tracking Functionality
# -----------------------------
def track_stock(stock_symbol, target_price, stop_loss, recipient_email, output_widget):
    """
    Tracks the stock price in real time and sends notifications when conditions are met.
    :param stock_symbol: Stock ticker symbol (e.g., "RELIANCE.NS" for Reliance Industries on NSE).
    :param target_price: Price target at which to alert (if the price goes above or equal).
    :param stop_loss: Stop loss price at which to alert (if the price goes below or equal).
    :param recipient_email: Email address to send notifications.
    :param output_widget: ipywidgets.Output instance to print status messages.
    """
    with output_widget:
        print("Starting stock tracking...")
    ticker = yf.Ticker(stock_symbol)
    notified = False
    while not notified:
        try:
            # Fetch the most recent price data (using a 1-minute interval for today)
            data = ticker.history(period="1d", interval="1m")
            if data.empty:
                with output_widget:
                    print("No data retrieved. Retrying...")
            else:
                current_price = data['Close'].iloc[-1]
                status_text = f"Current {stock_symbol} price: {current_price:.2f}"
                with output_widget:
                    print(status_text)

                # Check if target or stop loss conditions are met
                if current_price >= target_price:
                    with output_widget:
                        print(f"Target price reached: {current_price:.2f} >= {target_price:.2f}")
                        print("Sending target alert email...")
                    send_email_notification(recipient_email, stock_symbol, current_price, "TARGET")
                    with output_widget:
                        print(f"Alert: {stock_symbol} has hit the target price of {target_price:.2f}!")
                    notified = True
                elif current_price <= stop_loss:
                    with output_widget:
                        print(f"Stop loss triggered: {current_price:.2f} <= {stop_loss:.2f}")
                        print("Sending stop loss alert email...")
                    send_email_notification(recipient_email, stock_symbol, current_price, "STOP LOSS")
                    with output_widget:
                        print(f"Alert: {stock_symbol} has hit the stop loss price of {stop_loss:.2f}!")
                    notified = True
        except Exception as e:
            with output_widget:
                print("Error fetching price:", str(e))

        # Wait a few seconds before checking again (adjust as needed)
        time.sleep(5)

    with output_widget:
        print("Tracking stopped.")


# -----------------------------
# Create the ipywidgets Interface
# -----------------------------
# Input fields
stock_input = widgets.Text(
    value='RELIANCE.NS',
    description='Stock Symbol:',
    placeholder='e.g., RELIANCE.NS',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

target_input = widgets.Text(
    value='2500',
    description='Target Price:',
    placeholder='Enter target price',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

stop_input = widgets.Text(
    value='2400',
    description='Stop Loss Price:',
    placeholder='Enter stop loss price',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

email_input = widgets.Text(
    value='recipient@example.com',
    description='Notification Email:',
    placeholder='Enter email address',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

start_button = widgets.Button(
    description="Start Tracking",
    button_style='success'
)

# Output area to show status messages
output_area = widgets.Output()

# -----------------------------
# Callback function for the button
# -----------------------------
def on_start_button_clicked(b):
    with output_area:
        clear_output()  # Clear previous output messages

    stock_symbol = stock_input.value.strip()
    target_str = target_input.value.strip()
    stop_str = stop_input.value.strip()
    recipient_email = email_input.value.strip()

    # Basic input validation
    if not stock_symbol or not target_str or not stop_str or not recipient_email:
        with output_area:
            print("Error: Please fill in all fields.")
        return

    try:
        target_price = float(target_str)
        stop_loss = float(stop_str)
    except ValueError:
        with output_area:
            print("Error: Target Price and Stop Loss must be numbers.")
        return

    if stop_loss >= target_price:
        with output_area:
            print("Error: Stop Loss must be less than Target Price.")
        return

    # Inform the user that tracking has started
    with output_area:
        print("Initializing tracking for", stock_symbol)

    # Disable the button to prevent multiple tracking threads
    start_button.disabled = True

    # Start the tracking in a background thread so that the interface remains responsive
    tracking_thread = threading.Thread(
        target=track_stock,
        args=(stock_symbol, target_price, stop_loss, recipient_email, output_area),
        daemon=True
    )
    tracking_thread.start()

# Bind the button click event to the callback function
start_button.on_click(on_start_button_clicked)

# -----------------------------
# Display the Interface
# -----------------------------
interface = widgets.VBox([
    stock_input,
    target_input,
    stop_input,
    email_input,
    start_button,
    output_area
])

display(interface)


In [ ]:
# Install required packages if not already installed (uncomment if needed)
!pip install yfinance ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time
import datetime
import yfinance as yf
import smtplib
import matplotlib.pyplot as plt
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart



In [ ]:


# -----------------------------
# Email Configuration Settings
# -----------------------------
# Replace these with your email settings.
EMAIL_ADDRESS = "chehaktrehan21@gmail.com"        # Your sender email address (e.g., Gmail)
EMAIL_PASSWORD = "njwd obky invw kfjm"   # Your email password or app-specific password
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# -----------------------------
# Email Notification Function
# -----------------------------
def send_email_notification(to_email, stock_symbol, current_price, condition):
    """
    Sends an email notification.
    :param to_email: Recipient email address.
    :param stock_symbol: The stock symbol.
    :param current_price: The price at which the condition was met.
    :param condition: A string indicating whether it was a "TARGET" or "STOP LOSS" hit.
    """
    subject = f"Stock Alert: {stock_symbol} has hit {condition}!"
    body = (f"Hello,\n\n"
            f"This is an automated alert that {stock_symbol} has reached a price of {current_price:.2f},\n"
            f"which meets the {condition} condition that you set.\n\n"
            f"Regards,\nStock Alert System")

    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", str(e))


# -----------------------------
# Stock Tracking and Graph Updating Functionality
# -----------------------------
def track_stock(stock_symbol, target_price, stop_loss, recipient_email, text_output, graph_output):
    """
    Tracks the stock price in real time, updates a live graph, and sends notifications when conditions are met.
    :param stock_symbol: Stock ticker symbol (e.g., "RELIANCE.NS" for Reliance Industries on NSE).
    :param target_price: Price target at which to alert.
    :param stop_loss: Stop loss price at which to alert.
    :param recipient_email: Email address to send notifications.
    :param text_output: ipywidgets.Output instance to print text messages.
    :param graph_output: ipywidgets.Output instance to display the live graph.
    """
    with text_output:
        print("Starting stock tracking for", stock_symbol)

    ticker = yf.Ticker(stock_symbol)
    notified = False
    time_data = []
    price_data = []

    # Record the start time for the x-axis (using datetime objects)
    start_time = datetime.datetime.now()

    while not notified:
        try:
            # Fetch the most recent price data (using a 1-minute interval for today)
            data = ticker.history(period="1d", interval="1m")
            if data.empty:
                with text_output:
                    print("No data retrieved. Retrying...")
            else:
                # Get the latest price and current timestamp
                current_price = data['Close'].iloc[-1]
                current_time = datetime.datetime.now()

                # Append data for plotting
                time_data.append(current_time)
                price_data.append(current_price)

                status_text = f"Current {stock_symbol} price: {current_price:.2f} at {current_time.strftime('%H:%M:%S')}"
                with text_output:
                    print(status_text)

                # Update the graph
                with graph_output:
                    clear_output(wait=True)
                    plt.figure(figsize=(10, 4))
                    plt.plot(time_data, price_data, marker='o', linestyle='-')
                    plt.xlabel("Time")
                    plt.ylabel("Price")
                    plt.title(f"{stock_symbol} Price Over Time")
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.grid(True)
                    plt.show()

                # Check if target or stop loss conditions are met
                if current_price >= target_price:
                    with text_output:
                        print(f"Target price reached: {current_price:.2f} >= {target_price:.2f}")
                        print("Sending target alert email...")
                    send_email_notification(recipient_email, stock_symbol, current_price, "TARGET")
                    with text_output:
                        print(f"Alert: {stock_symbol} has hit the target price of {target_price:.2f}!")
                    notified = True
                elif current_price <= stop_loss:
                    with text_output:
                        print(f"Stop loss triggered: {current_price:.2f} <= {stop_loss:.2f}")
                        print("Sending stop loss alert email...")
                    send_email_notification(recipient_email, stock_symbol, current_price, "STOP LOSS")
                    with text_output:
                        print(f"Alert: {stock_symbol} has hit the stop loss price of {stop_loss:.2f}!")
                    notified = True
        except Exception as e:
            with text_output:
                print("Error fetching price:", str(e))

        # Wait a few seconds before checking again (adjust as needed)
        time.sleep(5)

    with text_output:
        print("Tracking stopped.")


# -----------------------------
# Create the ipywidgets Interface for Colab
# -----------------------------
# Input fields
stock_input = widgets.Text(
    value='RELIANCE.NS',
    description='Stock Symbol:',
    placeholder='e.g., RELIANCE.NS',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

target_input = widgets.Text(
    value='2500',
    description='Target Price:',
    placeholder='Enter target price',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

stop_input = widgets.Text(
    value='2400',
    description='Stop Loss Price:',
    placeholder='Enter stop loss price',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

email_input = widgets.Text(
    value='recipient@example.com',
    description='Notification Email:',
    placeholder='Enter email address',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

start_button = widgets.Button(
    description="Start Tracking",
    button_style='success'
)

# Output areas: one for text logs and one for the graph
text_output = widgets.Output()
graph_output = widgets.Output()

# -----------------------------
# Callback function for the Start Button
# -----------------------------
def on_start_button_clicked(b):
    with text_output:
        text_output.clear_output()  # Clear previous output messages
    with graph_output:
        graph_output.clear_output() # Clear previous graph

    stock_symbol = stock_input.value.strip()
    target_str = target_input.value.strip()
    stop_str = stop_input.value.strip()
    recipient_email = email_input.value.strip()

    # Basic input validation
    if not stock_symbol or not target_str or not stop_str or not recipient_email:
        with text_output:
            print("Error: Please fill in all fields.")
        return

    try:
        target_price = float(target_str)
        stop_loss = float(stop_str)
    except ValueError:
        with text_output:
            print("Error: Target Price and Stop Loss must be numbers.")
        return

    if stop_loss >= target_price:
        with text_output:
            print("Error: Stop Loss must be less than Target Price.")
        return

    with text_output:
        print("Initializing tracking for", stock_symbol)

    # Disable the start button to prevent multiple tracking threads
    start_button.disabled = True

    # Start the tracking in a background thread so that the interface remains responsive
    tracking_thread = threading.Thread(
        target=track_stock,
        args=(stock_symbol, target_price, stop_loss, recipient_email, text_output, graph_output),
        daemon=True
    )
    tracking_thread.start()

# Bind the button click event to the callback function
start_button.on_click(on_start_button_clicked)

# -----------------------------
# Display the Interface
# -----------------------------
interface = widgets.VBox([
    stock_input,
    target_input,
    stop_input,
    email_input,
    start_button,
    widgets.HTML(value="<h3>Tracking Log:</h3>"),
    text_output,
    widgets.HTML(value="<h3>Live Price Graph:</h3>"),
    graph_output
])


display(interface)


In [ ]:
import matplotlib.pyplot as plt
import datetime
from IPython.display import clear_output
import ipywidgets as widgets

# Store time and price data
time_data = []
price_data = []

def update_live_graph(current_time, current_price, graph_output):
    """
    Update the live graph with new time and price data.

    :param current_time: The current datetime object.
    :param current_price: The latest stock price.
    :param graph_output: The ipywidgets output widget to display the graph.
    """
    # Append the new data to the lists
    time_data.append(current_time)
    price_data.append(current_price)

    # Update the graph output
    with graph_output:
        clear_output(wait=True)  # Clears the output before showing the updated graph

        # Create a new plot
        plt.figure(figsize=(10, 4))
        plt.plot(time_data, price_data, marker='o', linestyle='-', color='b')
        plt.xlabel("Time")
        plt.ylabel("Price")
        plt.title("Live Stock Price Over Time")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()  # Ensures proper layout in Colab
        plt.show()

# Example usage (in your stock tracking loop):
# Assume current_price and current_time are updated in your tracking function

# For demonstration purposes, you can manually call the function here to see how it works
graph_output = widgets.Output()
display(graph_output)

# Simulate data update (this would be inside your stock tracking loop)
for i in range(5):  # Simulate 5 updates
    current_time = datetime.datetime.now()
    current_price = 2500 + (i * 5)  # Example: Increment price for testing
    update_live_graph(current_time, current_price, graph_output)
    time.sleep(1)  # Simulate waiting for the next price update


Output()